In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/shelter-animal-outcomes/train.csv.gz')
train
# label : OutcomeType

In [ ]:
train['OutcomeType'].value_counts()
# 다중분류문제!

In [ ]:
test = pd.read_csv('/kaggle/input/shelter-animal-outcomes/test.csv.gz')
test

In [ ]:
all_data = pd.concat([train,test])
all_data

In [ ]:
all_data['Name'] = all_data['Name'].notnull()  # .notnull() : 결측치 여부 확인

In [ ]:
# 시간 컬럼 생성
all_data['DateTime'] = pd.to_datetime(all_data['DateTime'])
all_data['year'] = all_data['DateTime'].dt.year
all_data['month'] = all_data['DateTime'].dt.month
all_data['week'] = all_data['DateTime'].dt.week
all_data['day'] = all_data['DateTime'].dt.day
all_data['hour'] = all_data['DateTime'].dt.hour
all_data['minute'] = all_data['DateTime'].dt.minute
all_data['time'] = (all_data['DateTime'].dt.date-all_data['DateTime'].dt.date.min()).dt.days
all_data

In [ ]:
# 특정 hour이 label에 영향 미치는지 그림으로 확인
# boxplot()은 범주형 1개컬럼 & 수치형 1개컬럼이 있어야 그릴수 있음
# 하지만 hour와 OutcomeType은 둘다 범주형
# --> countplot()
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16,8))
sns.countplot(all_data['hour'],hue=all_data['OutcomeType'])
# 저녁 5시 6시에 많음 --> 시간대 별로 y값에 영향을 충분히 미친다고 생각할 수 있음!

In [ ]:
# train/test 데이터셋에서 보면 AgeuponeOutcome의 단위가 애매하다!
# AgeuponOutcome 컬럼의 단위를 뭐라 설정할까?
all_data['AgeuponOutcome'].unique()
# 최소값이 1 day이므로 그에 맞게 설정하는 함수 만들기
# nan은?

In [ ]:
def age(x):
    if pd.isnull(x):
        return -1
    num = int(x.split()[0])   # 숫자와 문자 구분(공백이 디폴트이므로 따로 ' '처리할 필요 없음)
    if 'year' in x:
        return num * 365
    elif 'month' in x:
        return num * 30
    elif 'week' in x:
        return num * 7
    else : return num

all_data['AgeuponOutcome'] = all_data['AgeuponOutcome'].apply(age)
all_data

In [ ]:
print(train.columns)
print(test.columns)

In [ ]:
all_data_2 = all_data.drop(columns = ['DateTime', 'OutcomeType','AnimalID','OutcomeSubtype',
                                     'ID'])
all_data_2

In [ ]:
# 나머지 컬럼들 변형
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
c = all_data_2.columns[all_data_2.dtypes == object]

for i in c:
    all_data_2[i] = le.fit_transform(list(all_data_2[i]))
all_data_2

In [ ]:
all_data_2 = all_data_2.fillna(-1)

In [ ]:
train_2 = all_data_2[:len(train)]
test_2 = all_data_2[len(train):]

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1,n_estimators=1000)
rf.fit(train_2,train['OutcomeType'])

In [ ]:
result = rf.predict_proba(test_2)
result

In [ ]:
rf.classes_
# 위에 있는 확률과 아래 컬럼들은 같은 순서
# class의 순서는 알파벳 순서다!

In [ ]:
sub = pd.read_csv('/kaggle/input/shelter-animal-outcomes/sample_submission.csv.gz')
sub

In [ ]:
sub.iloc[:,1:] = result
sub

In [ ]:
sub.to_csv('rf_result.csv',index=False)
# cbc와 rf모델 점수 비교 --> 0.86 vs 1.5

In [ ]:
# 범주형 데이터만 읽어오기
np.where(all_data_2.dtypes !=float) # float이 아닌 요소들만 접근
# 소괄호가 2번있음 --> []접근!

In [ ]:
np.where(all_data_2.dtypes != float)[0]

In [ ]:
# CatBoost + float(숫자형)이 아닌 범주형 요소들만 학습
# 근데 사실상 전부 범주형 요소...
# 그냥 이렇게도 할 수 있다고 생각하자
from catboost import CatBoostClassifier
cb = CatBoostClassifier(verbose=100)
cb.fit(train_2,train['OutcomeType'],cat_features=np.where(all_data_2.dtypes != float)[0])

In [ ]:
# feature_importance : 변수중요도
cb.feature_importances_

In [ ]:
pd.Series(cb.feature_importances_,index=train_2.columns).sort_values(ascending=False)

# Name을 왜 중요하게 생각하나?
# 1) 진짜 의미가 있던지  2) 잘못 생각했던지

# 1) 특정부분(return to owner)관련 이름이 도움이 되기도 함
# --> 이름이 있는 경우와 없는 경우가 y값에 대해서 중요한 정보가 됨!
# 2) 운이 안좋으면(종류가 많으면) y값도 Name에 따라 바뀐다고 생각하는 경우

In [ ]:
# all_data['Name'] = all_data['Name'].isnull()

In [ ]:
import seaborn as sns
sns.countplot(all_data['Name'],hue=all_data['OutcomeType'])
# False/True에 따라서 y값이 많이 갈림!
# Name은 유효한 컬럼이다!

In [ ]:
result = cb.predict_proba(test_2)
result

In [ ]:
# catboost는 옵션설정 안해줘도 옵션 값에 덜 영향을 받는편!

In [ ]:
sub = pd.read_csv('/kaggle/input/shelter-animal-outcomes/sample_submission.csv.gz')
sub

In [ ]:
sub.iloc[:,1:] = result
sub.to_csv('cb_final_result.csv',index=False)

In [ ]:
# 날짜 컬럼 추가해서 예측력 높이기(위에서 진행)

In [ ]:
sub = pd.read_csv('/kaggle/input/shelter-animal-outcomes/sample_submission.csv.gz')
sub.iloc[:,1:]
sub.iloc[:,1:] = result
result
sub.to_csv('result1.csv',index=False)

In [ ]:
# Tree 모델 : 모델이 알아서 어떤 컬럼이 중요한지 알아내서 많이 학습
#            컬럼을 봤을때 수치형인지 범주형인지 알아서 판단 가능
#            but 선형적인 느낌을 받게 되면 수치형이라고 판단
#           --> 특정 컬럼은 범주형이라고 직접 지정해줄 수 있음!

In [ ]:
# 진행중인대회 베이스라인 잡아오기